In [8]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import plotly.express   as px
import cv2
import warnings
import plotly.graph_objects as go
from joblib import load
from skimage.feature import hog


In [3]:
warnings.simplefilter('ignore')

In [4]:
labels = []
images = []


In [5]:
root_path = 'sec1/testset'

In [6]:
for dirname, _,filenames in os.walk(root_path):
   
    labels = [file for file in filenames if file.endswith(".txt")]
    images = [file for file in filenames if file.endswith(".png")]

In [7]:
image_height = 1080
image_width = 1920

In [9]:
def get_bboxes(etiketler):
    bboxes=[]
    for etiket in etiketler:
        obje_sinifi = int(etiket[0])
        # Dikdörtgenin merkezine göre x, y konumlarını ve genişlik, yükseklik değerlerini al
        x = float(etiket[1])
        y = float(etiket[2])
        width = float(etiket[3])
        height = float(etiket[4])
        # Koordinatları ve boyutları belirtilen aralıkta gerçek değerlere dönüştür (örneğin, görüntü boyutuna göre)
        x = x * image_width
        y = y * image_height
        width = width * image_width
        height = height * image_height
        # Sonuçları bir listeye ekleyin
        bboxes.append([obje_sinifi, x, y, width, height])
    return bboxes


In [141]:
testdata = {}
for file_n in labels:

    with open(os.path.join(root_path, file_n), "r") as file:
        etiketler = [[i for i in line.strip().split()] for line in file.readlines()]
        bboxes = get_bboxes(etiketler)
        file_name = file_n[:-4] + ".png"
        testdata[file_name] =  bboxes 

In [74]:
def extract_hog_features(window):
    gray = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    resclaed_img = cv2.resize(gray, (40,30 ), interpolation=cv2.INTER_LINEAR)
    hog_features = hog(resclaed_img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    return hog_features    


def detect_vehicles(image, window_size, step_size, model):
    labels = []
    img_height, img_width = image.shape[:2]
    
    # Sliding window
    for y in range(0, img_height - window_size[1] + 1, step_size[1]):
        for x in range(0, img_width - window_size[0] + 1, step_size[0]):
            window = image[y:y+window_size[1], x:x+window_size[0]]
            # HOG features extraction
            hog_features = extract_hog_features(window)
            # Make prediction
            hog_features_resized = hog_features.reshape(1, -1) 
            prediction = model.predict(hog_features_resized)[0]
            # Create label
            label = [prediction, x + window_size[0] // 2, y + window_size[1] // 2, window_size[0], window_size[1]]
            labels.append(label)
    
    return labels

def predict_images(testdata,model,window_size,step_size):
    predictions = {}
    a = 0
    for image in testdata.keys():  
        if a == 100:
            break
        img_path = os.path.join(root_path, image )
        img = cv2.imread(img_path)
        predictions[image] = detect_vehicles(img, window_size, step_size, model)
        a+=1
    return predictions




def calculate_intersection_percentage(true_label, predicted_label):
    intersection_width = min(true_label[1] + true_label[3] / 2, predicted_label[1] + predicted_label[3] / 2) - max(true_label[1] - true_label[3] / 2, predicted_label[1] - predicted_label[3] / 2)
    intersection_height = min(true_label[2] + true_label[4] / 2, predicted_label[2] + predicted_label[4] / 2) - max(true_label[2] - true_label[4] / 2, predicted_label[2] - predicted_label[4] / 2)
    if intersection_width <= 0 or intersection_height <= 0:
        return 0
    intersection_area = intersection_width * intersection_height
    true_label_area = true_label[3] * true_label[4]
    return (intersection_area/ true_label_area) 


In [122]:
print(len(testdata))  

900


In [123]:
def save_predictions(image,etiket):
    print(3)
    img_path = os.path.join(root_path, image )
    img = cv2.imread(img_path)
    label,x, y, w, h = etiket
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    cropped_frame = img[int(y-h/2):int(y+h/2), int(x-w/2):int(x+w/2) ]  # Bounding box içindeki kırpılmış görüntü   
    plt.imshow(cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB))
    os.chdir("preds3")
    cv2.imwrite(f'{label}_{x}_{y}_'+image, cropped_frame)
    os.chdir("..")

evaluate predictions

In [132]:
len(predictions1)

100

In [139]:
def evaluate_prediction(predictions):
    cc = cm = cn = mc = mm = mn = nm = nc = nn = 0
    total_predict_num = 0
    for image in predictions.keys():
        df = pd.DataFrame(predictions[image])
        total_predict_num += len(df)
        df_ind = df[df.loc[:, 0] != 2].index
        df_last = df.iloc[df_ind]
        predicted_labels = df_last.values.tolist()
       
        true_labels = testdata[image].copy() 

        for predicted_label in predicted_labels:
            for true_label in true_labels:
                intersection_percentage = calculate_intersection_percentage(
                    true_label, predicted_label
                )
                if intersection_percentage > 0.2:
                
                    if true_label[0] == 1 and predicted_label[0] == 1:
                        cc += 1
                    elif true_label[0] == 0 and predicted_label[0] == 0:
                        mm += 1
                    elif true_label[0] == 1 and predicted_label[0] == 0:
                        mc += 1
                    elif true_label[0] == 0 and predicted_label[0] == 1:
                        cm += 1
                    true_labels.remove(true_label)
                    break
        for label in true_labels:
            if label[0] == 1:
                mn += 1
            else:
                cn += 1

        true_labels = testdata[image].copy() 
        updated_predicted_labels = predicted_labels.copy()
        # a copy to not change the orijinal list

        for true_label in true_labels:
            predicted_labels = updated_predicted_labels.copy()
            for predicted_label in predicted_labels:
                intersection_percentage = calculate_intersection_percentage(
                    true_label, predicted_label
                )
                if intersection_percentage > 0.2:
             

                    updated_predicted_labels.remove(predicted_label)
        for label in updated_predicted_labels:
            if label[0] == 1:
                nm += 1
            else:
                nc += 1

    nn = total_predict_num - sum([cc, cm, cn, mc, mm, mn, nm, nc])
    return [cc, cm, cn, mc, mm, mn, nm, nc, nn]

In [148]:
def calculate_accuracy(arr):
        cc,cm,cn,mc,mm,mn,nm,nc,nn = arr
        total_TP = sum([cc,mm,nn])
        total_instances = sum(arr)
        accuracy = total_TP / total_instances
        return accuracy

In [146]:
def print_confusion_matrix(arr):

    cc,cm,cn,mc,mm,mn,nm,nc,nn = arr
    confusion_matrix = np.array([
    [cc, cm, cn],  # Car sınıfı 
    [mc, mm, mn],  # Motorcycle 
    [nc, nm, nn]   # Non-vehicle 
    ])
    print("Confusion Matrix:\n", confusion_matrix)
    print("Accuracy: ",calculate_accuracy(arr))
 



In [ ]:
# Kaydedilmiş modeli yükleme
svm_model = load('svm_model.joblib')


In [ ]:
predictions1  = predict_images(testdata,svm_model,[40,30],[20,15])

In [ ]:
predictions2 = predict_images(testdata,svm_model,[80,60],[40,30])

In [ ]:
predictions3 = predict_images(testdata,svm_model,[120,90],[60,45])

In [ ]:
predictions4 = predict_images(testdata,svm_model,[180,120],[90,60])

In [149]:
print_confusion_matrix(evaluate_prediction(predictions4)  )
print_confusion_matrix(evaluate_prediction(predictions3)  )
print_confusion_matrix(evaluate_prediction(predictions2)  )
print_confusion_matrix(evaluate_prediction(predictions1)  )

Confusion Matrix:
 [[   13    16   290]
 [    7    26   227]
 [  140   150 33131]]
Accuracy:  0.9755882352941176
Confusion Matrix:
 [[   19    36   243]
 [   15    53   213]
 [  411   191 70119]]
Accuracy:  0.9844460028050491
Confusion Matrix:
 [[    56     79    171]
 [    22     82    169]
 [  1382    643 161573]]
Accuracy:  0.9849796256479287
Confusion Matrix:
 [[   102    120     70]
 [    51    142     94]
 [  6852   4193 662480]]
Accuracy:  0.9831183318894414


In [153]:
# Kaydedilmiş modeli yükleme
xgb_model = load('xgb_model.joblib')


In [154]:
predictions1_XGB = predict_images(testdata,xgb_model,[40,30],[20,15])
predictions2_XGB = predict_images(testdata,xgb_model,[80,60],[40,30])
predictions3_XGB = predict_images(testdata,xgb_model,[120,90],[60,45])
predictions4_XGB = predict_images(testdata,xgb_model,[180,120],[90,60])

In [155]:
print_confusion_matrix(evaluate_prediction(predictions4_XGB)  )
print_confusion_matrix(evaluate_prediction(predictions3_XGB)  )
print_confusion_matrix(evaluate_prediction(predictions2_XGB)  )
print_confusion_matrix(evaluate_prediction(predictions1_XGB)  )

Confusion Matrix:
 [[    9     8   309]
 [    1    15   237]
 [   29    31 33361]]
Accuracy:  0.9819117647058824
Confusion Matrix:
 [[   11    30   270]
 [    5    32   231]
 [   96    86 70539]]
Accuracy:  0.9899298737727911
Confusion Matrix:
 [[    50     65    159]
 [    18    108    179]
 [   184    259 163478]]
Accuracy:  0.9947477203647417
Confusion Matrix:
 [[   100     85     81]
 [    20    166    127]
 [   969   1618 671334]]
Accuracy:  0.995700518902891


In [156]:
# Kaydedilmiş modeli yükleme
mlp_model = load('mlp_model.joblib')


In [157]:
predictions1_MLP = predict_images(testdata,mlp_model,[40,30],[20,15])
predictions2_MLP = predict_images(testdata,mlp_model,[80,60],[40,30])
predictions3_MLP = predict_images(testdata,mlp_model,[120,90],[60,45])
predictions4_MLP = predict_images(testdata,mlp_model,[180,120],[90,60])

In [158]:
print_confusion_matrix(evaluate_prediction(predictions4_MLP)  )
print_confusion_matrix(evaluate_prediction(predictions3_MLP)  )
print_confusion_matrix(evaluate_prediction(predictions2_MLP)  )
print_confusion_matrix(evaluate_prediction(predictions1_MLP)  )

Confusion Matrix:
 [[    4    10   301]
 [    3    21   240]
 [   53    29 33339]]
Accuracy:  0.9812941176470589
Confusion Matrix:
 [[   13    20   284]
 [   10    28   224]
 [  217    68 70436]]
Accuracy:  0.9884572230014025
Confusion Matrix:
 [[    37     37    210]
 [    14     85    196]
 [   463    242 163216]]
Accuracy:  0.9929361702127659
Confusion Matrix:
 [[    84     60     98]
 [    24    174    139]
 [  2117   1685 670119]]
Accuracy:  0.993887323943662
